In [ ]:

%pip install -U anyscale -q
%load_ext autoreload
%autoreload 2

import os
import ray
import json
import datasets
from datasets import DatasetDict, load_dataset
import anyscale
from anyscale.llm.dataset import Dataset as AnyscaleDataset
import yaml
from rich import print
from src.utils import SYSTEM_CONTENT, to_llm_schema, get_dataset_file_path, update_datasets_in_fine_tuning_config, download_files_from_remote

# Initialize HF token
# assert ~/default/.HF_TOKEN exists
assert os.path.exists(os.path.expanduser('~/default/.HF_TOKEN')), (
    'Please create ~/default/.HF_TOKEN with your Hugging Face token\n'
    'echo "your_token" > ~/default/.HF_TOKEN'
)
HF_TOKEN = open(os.path.expanduser('~/default/.HF_TOKEN')).read().strip()

os.environ['HF_TOKEN'] = HF_TOKEN
ray.shutdown()
ray.init(runtime_env={'env_vars': {'HF_TOKEN': HF_TOKEN}})
LLAMA_3_SERVE_CONFIG_PATH = 'deploy/services/model_config/meta-llama--Meta-Llama-3-8B-Instruct.yaml'
config = yaml.safe_load(open(LLAMA_3_SERVE_CONFIG_PATH))
config['runtime_env']['env_vars']['HUGGING_FACE_HUB_TOKEN'] = HF_TOKEN
with open(LLAMA_3_SERVE_CONFIG_PATH, 'w') as f:
    yaml.safe_dump(config, f)


ray.data.DataContext.get_current().enable_progress_bars = False
ray.data.DataContext.get_current().print_on_execution_start = False
datasets.disable_progress_bars()

In [ ]:
# Deploy Service
!anyscale service deploy -f deploy/services/serve.yaml

# End-to-end LLM Workflows

In this guide, we'll learn how to run an end-to-end LLM workflow. We separate this into four steps:

1. **Data preprocessing**
2. **Fine-tuning**
3. **Serving**
4. **Evaluation**

**Objective**: Have an LLM convert unstructured text inputs about video games into structured text outputs.

## 0. Problem Statement

Imagine we are trying to convert an unstructured sentence into structured output. Take the problem statement below.

In [ ]:
print(SYSTEM_CONTENT)

Given a target sentence, construct the underlying meaning representation of the input sentence as a 
single function with attributes and attribute values.

This function should describe the target string accurately and the function must be one of the following

['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 'request_explanation',
'recommend', 'request_attribute'].
    
The attributes must be one of the following:

['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 'player_perspective',
'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 'has_mac_release', 'specifier'].

Let's first query a base model, Meta's Llama 3-8B model, to see how it performs on this task.

In [ ]:
from src.utils import query

response = query(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    prompt="Dirt: Showdown from 2012 is a sport racing game for the PlayStation, Xbox, PC "
    "rated E 10+ (for Everyone 10 and Older). It's not available on Steam, Linux, or Mac."
)
print(response)

Not great, right? It's slow and verbose. We were looking for an output like below:

```python
inform(
    name["Dirt: Showdown"],
    release_year[2012],
    esrb["E 10+ (for Everyone 10 and Older)"],
    genres["driving/racing", "sport"],
    platforms["PlayStation", "Xbox", "PC"],
    available_on_steam[False],
    has_linux_release[False],
    has_mac_release[False]
)
```

## 1. Data Preprocessing

We can use Ray Data and Anyscale Datasets to transform a dataset we have about video games (VIGGO) into a LLM conversation format (`system` / `user` / `assistant`) that the model can understand. 

<img src="assets/data-overview.png" width=500>

### Dataset

In [ ]:
dataset: DatasetDict = load_dataset("GEM/viggo", trust_remote_code=True)  # type: ignore

def get_dataset(split: str) -> AnyscaleDataset:
    ray_dataset = ray.data.from_items(dataset[split]).map(to_llm_schema)
    with get_dataset_file_path(ray_dataset) as dataset_file_path:
        anyscale_dataset = anyscale.llm.dataset.upload(
            dataset_file_path,
            name=f"viggo/{split}",
        )
    return anyscale_dataset

# Split the dataset into train, validation, and test sets
train_dataset = get_dataset("train")
val_dataset = get_dataset("validation")
test_dataset = get_dataset("test")

## 2. Fine-tuning

Next, we'll fine-tune a large language model (LLM) using our dataset with LLMForge, Ray Train, and an Anyscale Job.

We'll be fine-tuning Meta's Llama 3-8B model, which is the model we queried in the problem statement.

<img src="assets/train-overview.png" width=500>

In [ ]:
from anyscale.job import JobConfig

update_datasets_in_fine_tuning_config("configs/training/lora/llama-3-8b.yaml", train_dataset, val_dataset)
job_config = JobConfig.from_yaml("deploy/jobs/ft.yaml")
job_id = anyscale.job.submit(job_config)  # type: ignore

## 3. Serving

Now, let's query our fine-tuned model. Our fine-tuned model is hosted on an Anyscale Service that uses RayLLM and Ray Serve.

<img src="assets/online-overview.png" width=500>

In [ ]:
job_id = "prodjob_lgcmhahdme45fc4hbyah82m6a7"  # e2e-llm-workflows

fine_tuned_model = anyscale.llm.model.get(job_id=job_id)  # type: ignore
response = query(
    fine_tuned_model.id,
    prompt="Dirt: Showdown from 2012 is a sport racing game for the PlayStation, Xbox, PC "
    "rated E 10+ (for Everyone 10 and Older). It's not available on Steam, Linux, or Mac."
)
print(response)

inform(name[Dirt: Showdown], release_year[2012], esrb[E 10+ (for Everyone 10 and Older)], genres, 
platforms[PlayStation, Xbox, PC], available_on_steam, has_linux_release, has_mac_release)

```python
inform(
    name["Dirt: Showdown"],
    release_year[2012],
    esrb["E 10+ (for Everyone 10 and Older)"],
    genres["driving/racing", "sport"],
    platforms["PlayStation", "Xbox", "PC"],
    available_on_steam[False],
    has_linux_release[False],
    has_mac_release[False]
)
```

See how much better the output is?

## 4. Evaluation

We can evaluate our fine-tuned LLM to see how well it performs on our task. We'll start by performing offline batch inference where we will use our fine-tuned model to generate the outputs.

<img src="assets/offline-overview.png" width=500>

### Load test data

In [ ]:
from transformers import AutoTokenizer

# Load test set for eval
test_data = ray.data.read_json(test_dataset.storage_uri).take_all()
test_inputs = [
    [message for message in item['messages'] if message['role'] != 'assistant']
    for item in test_data
]
test_outputs = [
    [message for message in item['messages'] if message['role'] == 'assistant']
    for item in test_data
]

# Model and tokenizer
HF_MODEL = 'meta-llama/Meta-Llama-3-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL)

# Download artifacts
local_dir = f'/mnt/cluster_storage/{fine_tuned_model.id}'  # Storage accessible by head and worker nodes
download_files_from_remote(fine_tuned_model.storage_uri, local_dir)

# Extract chat template used during fine-tuning
tokenizer_config = json.load(open(os.path.join(local_dir, 'tokenizer_config.json')))
chat_template = tokenizer_config['chat_template']

### Chat template

When we fine-tuned our model, special tokens (ex. beginning/end of text, etc.) were automatically added to our inputs. We want to apply the same special tokens to our inputs prior to generating outputs using our tuned model. Luckily, the chat template to apply to our inputs (and add those tokens) is readily available inside our tuned model's `tokenizer_config.json` file. We can use our tokenizer to apply this template to our inputs.

In [13]:
# Apply chat template
test_input_prompts = [{'inputs': tokenizer.apply_chat_template(
    conversation=inputs,
    chat_template=chat_template,
    add_generation_prompt=True,
    tokenize=False,
    return_tensors='np'), 'outputs': outputs} for inputs, outputs in zip(test_inputs, test_outputs)]
test_input_prompts_ds = ray.data.from_items(test_input_prompts)
print (test_input_prompts_ds.take(1))

2024-09-28 01:13:23,840	INFO dataset.py:2416 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.


[
    {
        'inputs': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nGiven a target sentence, 
construct the underlying meaning representation of the input sentence as a \nsingle function with attributes and 
attribute values.\n\nThis function should describe the target string accurately and the function must be one of the
following\n\n['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 
'request_explanation',\n'recommend', 'request_attribute'].\n    \nThe attributes must be one of the 
following:\n\n['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 
'player_perspective',\n'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 
'has_mac_release', 'specifier'].<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI remember you saying you 
found Little Big Adventure to be average. Are you not usually that into single-player games on 
PlayStation?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        'outputs': [
            {
                'content': 'verify_attribute(name[Little Big Adventure], rating[average], has_multiplayer[no], 
platforms[PlayStation])',
                'role': 'assistant'
            }
        ]
    }
]

### Batch inference

We will use [vLLM](https://github.com/vllm-project/vllm)'s offline LLM class to load the model and use it for inference. We can easily load our LoRA weights and merge them with the base model (just pass in `lora_path`). And we'll wrap all of this functionality in a class that we can pass to [ray.data.Dataset.map_batches](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.map_batches.html) to apply batch inference at scale.

<img src="assets/offline-detailed.png" width=750>

In [16]:
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest

Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')


In [14]:
class LLMPredictor:
    def __init__(self, hf_model, sampling_params, lora_path=None):
        self.llm = LLM(model=hf_model, enable_lora=bool(lora_path))
        self.sampling_params = sampling_params
        self.lora_path = lora_path

    def __call__(self, batch):
        if not self.lora_path:
            outputs = self.llm.generate(
                prompts=batch['inputs'],
                sampling_params=self.sampling_params)
        else:
            outputs = self.llm.generate(
                prompts=batch['inputs'],
                sampling_params=self.sampling_params,
                lora_request=LoRARequest('lora_adapter', 1, self.lora_path))
        inputs = []
        generated_outputs = []
        for output in outputs:
            inputs.append(output.prompt)
            generated_outputs.append(' '.join([o.text for o in output.outputs]))
        return {
            'prompt': inputs,
            'expected_output': batch['outputs'],
            'generated_text': generated_outputs,
        }

During our data preprocessing template, we used the default compute strategy with `map_batches`. But this time we'll specify a custom compute strategy (`concurrency`, `num_gpus`, `batch_size` and `accelerator_type`).

In [17]:
# Fine-tuned model
hf_model = 'meta-llama/Meta-Llama-3-8B-Instruct'
sampling_params = SamplingParams(temperature=0, max_tokens=2048)
ft_pred_ds = test_input_prompts_ds.map_batches(
    LLMPredictor,
    concurrency=4,  # number of LLM instances
    num_gpus=1,     # GPUs per LLM instance
    batch_size=10,  # maximize until OOM, if OOM then decrease batch_size
    fn_constructor_kwargs={
        'hf_model': hf_model,
        'sampling_params': sampling_params,
        'lora_path': local_dir,
    },
    accelerator_type='A10G',  # A10G or L4
)

In [21]:
# Batch inference will take ~4 minutes
ft_pred = ft_pred_ds.take_all()
ft_pred[3]

{'prompt': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Given a target sentence, construct the underlying meaning representation of the input sentence as a 
single function with attributes and attribute values.

This function should describe the target string accurately and the function must be one of the following

['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 'request_explanation',
'recommend', 'request_attribute'].
    
The attributes must be one of the following:

['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 'player_perspective',
'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 'has_mac_release', 
'specifier'].<|eot_id|><|start_header_id|>user<|end_header_id|>

Might & Magic: Heroes VI was a barely adequate PC game. From the bird view perspective to the gameplay, everything 
felt very by the numbers.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

",
 'expected_output': array([{'content': 'give_opinion(name[Might & Magic: Heroes VI], rating, player_perspective, 
platforms[PC])', 'role': 'assistant'}],
       dtype=object),
 'generated_text': 'give_opinion(name[Might & Magic: Heroes VI], rating, player_perspective, platforms[PC])'}

### Evaluation

In [23]:
# Exact match (strict!)
matches = 0
mismatches = []
for item in ft_pred:
    if item['expected_output'][0]['content'] == item['generated_text'].split('<|eot_id|>')[0]:
        matches += 1
    else:
        mismatches.append(item)
print("Percentage of exact matches:", f"{(matches / float(len(ft_pred)) * 100):.2f}%")

Percentage of exact matches: 77.38%

**Note**: you can train for more epochs (`num_epochs: 10`) to further improve the performance.

Even our mismatches are not too far off and sometimes it might be worth a closer look because the dataset itself might have a few errors that the model may have identified.

In [24]:
# Inspect a few of the mismatches
print(mismatches[0:2])

[
    {
        'prompt': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nGiven a target sentence, 
construct the underlying meaning representation of the input sentence as a \nsingle function with attributes and 
attribute values.\n\nThis function should describe the target string accurately and the function must be one of the
following\n\n['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 
'request_explanation',\n'recommend', 'request_attribute'].\n    \nThe attributes must be one of the 
following:\n\n['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 
'player_perspective',\n'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 
'has_mac_release', 'specifier'].<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat would you say was the 
weirdest game you've played?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        'expected_output': array([{'content': 'request(specifier[weirdest])', 'role': 'assistant'}],
      dtype=object),
        'generated_text': 'request(specifier[weird])'
    },
    {
        'prompt': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nGiven a target sentence, 
construct the underlying meaning representation of the input sentence as a \nsingle function with attributes and 
attribute values.\n\nThis function should describe the target string accurately and the function must be one of the
following\n\n['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 
'request_explanation',\n'recommend', 'request_attribute'].\n    \nThe attributes must be one of the 
following:\n\n['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 
'player_perspective',\n'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 
'has_mac_release', 'specifier'].<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat's the weirdest game you 
have ever heard of?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        'expected_output': array([{'content': 'request(specifier[weirdest])', 'role': 'assistant'}],
      dtype=object),
        'generated_text': 'request(specifier[weird])'
    }
]

## End-to-End Integration

<img src="assets/ai-platform.png" width=650>

## Next steps

We have a lot more guides that address more nuanced use cases:

Fine-tuning:
- [Control over 50+ hyperparameters](https://docs.anyscale.com/llms/finetuning/guides/modify_hyperparams/)
- [Fine-tune any HF model](https://docs.anyscale.com/llms/finetuning/guides/bring_any_hf_model/)
- [Full-parameter or LoRA fine-tuning](https://docs.anyscale.com/llms/finetuning/guides/lora_vs_full_param/)
- [Classification fine-tuning / Routing](https://www.anyscale.com/blog/building-an-llm-router-for-high-quality-and-cost-effective-responses)
- [Function calling fine-tuning](https://github.com/anyscale/templates/blob/main/templates/fine-tune-llm_v2/end-to-end-examples/fine-tune-function-calling/README.ipynb)
- [Longer context fine-tuning](https://www.anyscale.com/blog/fine-tuning-llms-for-longer-context-and-better-rag-systems)
- [Continued fine-tuning from checkpoint](https://github.com/anyscale/templates/tree/main/templates/fine-tune-llm_v2/cookbooks/continue_from_checkpoint)
- Training on more available hardware (ex. A10s) with model parallelism
- [End-to-end LLM workflows (including batch data processing, batch inference)](https://www.anyscale.com/blog/end-to-end-llm-workflows-guide)
- Distillation (Coming in <2 weeks)

Serving:
- [Deploy with autoscaling + optimize for latency vs. throughput](https://docs.anyscale.com/examples/deploy-llms/)
- [Serving multiple LoRA adapters](https://docs.anyscale.com/llms/serving/guides/multi_lora/)
- [Migration from OpenAI](https://docs.anyscale.com/llms/serving/guides/openai_to_oss/)
- [Spot to on-demand fallback (vice versa)](https://docs.anyscale.com/1.0.0/configure/compute-configs/ondemand-to-spot-fallback/)
- [Batch inference with vLLM](https://docs.anyscale.com/examples/batch-llm/)

And more!
- [Batch text embeddings with Ray data](https://github.com/anyscale/templates/tree/main/templates/text-embeddings)
- [Production RAG applications](https://www.anyscale.com/blog/a-comprehensive-guide-for-building-rag-based-llm-applications-part-1)
- [Router](https://github.com/anyscale/llm-router) between different models (base, fine-tuned, closed-source) to optimize for cost and quality
- Stable diffusion [fine-tuning](https://github.com/anyscale/templates/tree/main/templates/fine-tune-stable-diffusion) and [serving](https://github.com/anyscale/templates/tree/main/templates/serve-stable-diffusion)

And if you're interested in using our hosted Anyscale or connecting it to your own cloud, reach out to us at [Anyscale](https://www.anyscale.com/get-started?utm_source=goku). And follow us on [Twitter](https://x.com/anyscalecompute) and [LinkedIn](https://www.linkedin.com/company/joinanyscale/) for more real-time updates on new features!

## Clean up

In [ ]:
# Clean up
!python src/clear_cell_nums.py
!find . | grep -E ".ipynb_checkpoints" | xargs rm -rf
!find . | grep -E "(__pycache__|\.pyc|\.pyo)" | xargs rm -rf
!rm -rf __pycache__ data .HF_TOKEN deploy/services